# TP 2 - NLP - Bot


<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


### Alumno: Domenje Carlos R.

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

__IMPORTANTE__: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.

### Librerias a utilizar

In [1]:
!pip install spacy==3.3 --quiet
!pip install spacy-stanza==1.0.2 --quiet
!pip install lxml

In [2]:

import string
import re
import unicodedata
import urllib.request

import nltk
import warnings
warnings.filterwarnings('ignore')

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import stanza
import spacy_stanza

In [3]:

# Descargamos el diccionario en español y armar el pipeline de NLP con spacy
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")


2022-11-13 22:37:18 INFO: Downloading default packages for language: es (Spanish) ...
2022-11-13 22:37:19 INFO: File exists: /Users/carlosdomenje/stanza_resources/es/default.zip
2022-11-13 22:37:22 INFO: Finished downloading models and saved to /Users/carlosdomenje/stanza_resources.
2022-11-13 22:37:22 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-11-13 22:37:23 INFO: Loading these models for language: es (Spanish):
| Processor    | Package  |
---------------------------
| tokenize     | ancora   |
| mwt          | ancora   |
| pos          | ancora   |
| lemma        | ancora   |
| depparse     | ancora   |
| sentiment    | tass2020 |
| constituency | combined |
| ner          | conll02  |

2022-11-13 22:37:23 INFO: Use device: cpu
2022-11-13 22:37:23 INFO: Loading: tokenize
2022-11-13 22:37:23 INFO: Loading: mwt
2022-11-13 22:37:23 INFO: Loading: pos
2022-11-13 22:37:23 INFO: Loading: lemma
2022-11-13 22:37:23 INFO: Loading: depparse
2022-11-13 22:37:23 INFO: Loading: sentiment
2022-11-13 22:37:24 INFO: Loading: constituency
2022-11-13 22:37:24 INFO: Loading: ner
2022-11-13 22:37:24 INFO: Done loading processors!


### Datos para crear el BOT

Se toma un documento de wikipedia donde explica que es la inteligencia artificial.

URL: https://es.wikipedia.org/wiki/Inteligencia_artificial

In [4]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Inteligencia_artificial')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

print(article_text)


la inteligencia artificial es, en ciencias de la computación, la disciplina que intenta replicar y desarrollar la inteligencia y sus procesos implícitos a través de computadoras. no existe un acuerdo sobre la definición completa de inteligencia artificial, pero se han seguido cuatro enfoques: dos centrados en los humanos (sistemas que piensan como humanos, y sistemas que actúan como humanos) y dos centrados en torno a la racionalidad (sistemas que piensan racionalmente y sistemas que actúan racionalmente). comenzó poco después de la segunda guerra mundial, y el nombre se acuñó en 1956 en la conferencia de dartmouth por el informático john mccarthy.
la inteligencia artificial abarca en la actualidad una gran variedad de subcampos, que van desde áreas de propósito general, como el aprendizaje y la percepción, a otras más específicas como el juego de ajedrez, la demostración de teoremas matemáticos, la escritura de poesía y el diagnóstico de enfermedades. la inteligencia artificial sintet

In [5]:
print("Total de caracteres:", len(article_text))

Total de caracteres: 25274


In [6]:
lista_caracteres = list(set(article_text))
print("Lista de caracteres del texto: ",lista_caracteres)

Lista de caracteres del texto:  ['d', 'm', 'k', ':', '-', 'a', 't', 'á', 'p', 'í', '4', 'i', 'g', '»', 'r', 'o', '\u200b', 'ü', '9', 'h', '!', '.', '\xa0', '1', 'v', '3', 's', '(', 'é', '\n', 'u', '[', ')', ',', ';', 'j', 'n', 'l', 'w', '6', 'x', 'y', '–', 'q', 'z', '5', '0', ' ', 'c', 'ñ', ']', '7', 'b', 'f', '8', 'ú', '¡', '«', 'ó', '2', 'e']


### Preprocesamiento de texto.

Se remueven los caracteres especiales, acentos, espacios y saltos de linea

In [7]:
def clean_text(text):    
    # sacar tildes de las palabras
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar los suscripts
    text = re.sub("[\[].*?[\]]", "", text)
    text = text.replace('\u200b', '')
    
    # caracteres especiales
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    return text

In [8]:
text = clean_text(article_text)
text

'la inteligencia artificial es, en ciencias de la computacion, la disciplina que intenta replicar y desarrollar la inteligencia y sus procesos implicitos a traves de computadoras. no existe un acuerdo sobre la definicion completa de inteligencia artificial, pero se han seguido cuatro enfoques: dos centrados en los humanos (sistemas que piensan como humanos, y sistemas que actuan como humanos) y dos centrados en torno a la racionalidad (sistemas que piensan racionalmente y sistemas que actuan racionalmente). comenzo poco despues de la segunda guerra mundial, y el nombre se acuno en 1956 en la conferencia de dartmouth por el informatico john mccarthy. la inteligencia artificial abarca en la actualidad una gran variedad de subcampos, que van desde areas de proposito general, como el aprendizaje y la percepcion, a otras mas especificas como el juego de ajedrez, la demostracion de teoremas matematicos, la escritura de poesia y el diagnostico de enfermedades. la inteligencia artificial sinte

In [9]:
print("Total caracteres:", len(text))
char_list = list(set(text))
print("Listado de caracteres: ",char_list)

Total caracteres: 24975
Listado de caracteres:  ['d', 'm', 'k', ':', '-', 'a', 't', 'p', '4', 'i', 'g', 'r', 'o', '9', 'h', '!', '.', '1', 'v', '3', 's', '(', 'u', ')', ',', ';', 'j', 'n', 'l', 'w', '6', 'x', 'y', 'q', 'z', '5', '0', ' ', 'c', '7', 'b', 'f', '8', '2', 'e']


In [10]:
# Generacion del CORPUS
nltk.download()
corpus = nltk.sent_tokenize(text)
corpus

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


['la inteligencia artificial es, en ciencias de la computacion, la disciplina que intenta replicar y desarrollar la inteligencia y sus procesos implicitos a traves de computadoras.',
 'no existe un acuerdo sobre la definicion completa de inteligencia artificial, pero se han seguido cuatro enfoques: dos centrados en los humanos (sistemas que piensan como humanos, y sistemas que actuan como humanos) y dos centrados en torno a la racionalidad (sistemas que piensan racionalmente y sistemas que actuan racionalmente).',
 'comenzo poco despues de la segunda guerra mundial, y el nombre se acuno en 1956 en la conferencia de dartmouth por el informatico john mccarthy.',
 'la inteligencia artificial abarca en la actualidad una gran variedad de subcampos, que van desde areas de proposito general, como el aprendizaje y la percepcion, a otras mas especificas como el juego de ajedrez, la demostracion de teoremas matematicos, la escritura de poesia y el diagnostico de enfermedades.',
 'la inteligencia

### Procesamiento de texto aplicando spacy

Para esta función se aplica la tokenizacion, lematizacion. Se remueven los stopwords y puntuacion.

In [11]:
def process_text(text):
    doc = nlp(text)
    
    # Tokenization y lemmatization
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    
    # Filtrado de stop words
    filtered_sentence =[]
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    # Filtrado de puntuacion
    puntuaction = string.punctuation + "¿¡"
    filtered_sentence = [w for w in filtered_sentence if w not in puntuaction]

    return filtered_sentence

In [12]:
filter_words = process_text(text)
print("Palabras procesadas:\n",filter_words)

Palabras procesadas:
 ['inteligencia', 'artificial', 'ciencia', 'computacion', 'disciplina', 'intentar', 'replicar', 'desarrollar', 'inteligencia', 'proceso', 'implicito', 'trave', 'computadora', 'existir', 'definicion', 'completo', 'inteligencia', 'artificial', 'seguir', 'enfoque', 'centrado', 'humanos', 'sistema', 'pensar', 'humano', 'sistema', 'actuar', 'humanos', 'centrado', 'torno', 'racionalidad', 'sistema', 'pensar', 'racionalmente', 'sistema', 'actuar', 'racionalmente', 'comer', 'guerra', 'mundial', 'nombre', 'acunir', '1956', 'conferencia', 'dartmouth', 'informatico', 'john', 'mccarthy', 'inteligencia', 'artificial', 'abarcar', 'actualidad', 'variedad', 'subcampo', 'area', 'proposito', 'general', 'aprendizaje', 'percepcion', 'especifico', 'juego', 'ajedrez', 'demostracion', 'teorema', 'matematico', 'escritura', 'poesia', 'diagnostico', 'enfermedad', 'inteligencia', 'artificial', 'sintetizar', 'automatizar', 'tarea', 'principio', 'intelectual', 'potencialmente', 'relevante', 'c

### Vector TF-IDF
Se utilizan las stop words de spacy.

In [13]:
from spacy.lang.es.stop_words import STOP_WORDS as STOP_WORDS_spacy
spacy_stopwords = STOP_WORDS_spacy

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    corpus.append(user_input)

    word_vector = TfidfVectorizer(tokenizer=process_text, stop_words=spacy_stopwords)

    all_word_vectors = word_vector.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "Por favor, prueba con otra consulta..."
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

In [15]:
qst = "¿Que abarca la inteligencia artificial?" 
resp = generate_response(qst,corpus)
print("Pregunta: ",qst)
print("\nRespuesta: ",resp)

Pregunta:  ¿Que abarca la inteligencia artificial?

Respuesta:  la inteligencia artificial abarca en la actualidad una gran variedad de subcampos, que van desde areas de proposito general, como el aprendizaje y la percepcion, a otras mas especificas como el juego de ajedrez, la demostracion de teoremas matematicos, la escritura de poesia y el diagnostico de enfermedades.


In [16]:
pregunta = "¿Que es el aprendizaje automatico?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿Que es el aprendizaje automatico?

Respuesta:  449455 el aprendizaje automatico es un concepto fundamental de la investigacion de la inteligencia artificial desde el inicio del campo; consiste en el estudio de algoritmos informaticos que mejoran automaticamente a traves de la experiencia.


In [17]:
pregunta = "¿Que es el procesamiento del lenguaje natural?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿Que es el procesamiento del lenguaje natural?

Respuesta:  el procesamiento del lenguaje natural permite a las maquinas leer y comprender el lenguaje humano.


In [18]:
pregunta = "¿Cuales son las aplicaciones de la inteligencia artificial?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿Cuales son las aplicaciones de la inteligencia artificial?

Respuesta:  las aplicaciones incluyen reconocimiento de voz, reconocimiento facial y reconocimiento de objetos.


In [19]:
pregunta = "¿Cuales son las peliculas basadas en inteligencia artificial?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿Cuales son las peliculas basadas en inteligencia artificial?

Respuesta:  con el desarrollo de la tecnologia de inteligencia artificial, muchas companias de software como el aprendizaje profundo y el procesamiento del lenguaje natural han comenzado a producirse y la cantidad de peliculas sobre inteligencia artificial ha aumentado.


In [20]:
pregunta = "¿Que es la electronica?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿Que es la electronica?

Respuesta:  Por favor, prueba con otra consulta...
